1. Cargar y limpiar datos

In [11]:
# 1.1 Cargar datos de entrenamiento
import pandas as pd  #  type: ignore

data_train = pd.read_csv("../files/input/train_data.csv.zip", index_col=False, compression="zip")
data_test = pd.read_csv("../files/input/test_data.csv.zip", index_col=False, compression="zip")

In [ ]:
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.

data_train["Age"] = 2021 - data_train["Year"]
data_test["Age"] = 2021 - data_test["Year"]

data_train = data_train.drop(columns=["Year", "Car_Name"])
data_test = data_test.drop(columns=["Year", "Car_Name"])

Paso 2. Divida los datasets en x_train, y_train, x_test, y_test.

In [13]:
x_train = data_train.drop(columns=["Present_Price"])
y_train = data_train["Present_Price"]

x_test = data_test.drop(columns=["Present_Price"])
y_test = data_test["Present_Price"]

Paso 3: Creación del pipeline

In [ ]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

categorical_features = ["Fuel_Type", "Selling_type", "Transmission"]
numerical_features = [col for col in x_train.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", MinMaxScaler(), numerical_features),
    ],
    remainder=MinMaxScaler(),
)

pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("feature_selection", SelectKBest(f_regression)),
        ("classifier", LinearRegression()),
    ]
)


# Ajustar el pipeline a los datos de entrenamiento
pipeline.fit(x_train, y_train)

# Evaluar el modelo en el conjunto de prueba
print("Modelo entrenado. Precisión en test:", pipeline.score(x_test, y_test))



Paso 4. Definir los hiperparámetros

In [ ]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.


from sklearn.model_selection import GridSearchCV


param_grid = {
    "feature_selection__k": range(1, 12),
}


grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring="neg_mean_absolute_error", n_jobs=-1)

grid_search.fit(x_train, y_train)

Paso 5. Guardar el modelo

In [16]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.
import pickle
import gzip

# Guardar el modelo comprimido con gzip
with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
    pickle.dump(grid_search, file)   


Paso 6. Calcular la matriz

In [17]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,median_absolute_error

# Predecir en los conjuntos de entrenamiento y prueba
# y_train_pred = grid_search.best_estimator_.predict(x_train)
# y_test_pred = grid_search.best_estimator_.predict(x_test)

# Calcular las métricas
metrics_train = {
    "type": "metrics",
    "dataset": "train",
    "r2": float(r2_score(y_train, grid_search.predict(x_train))),
    "mse": float(mean_squared_error(y_train, grid_search.predict(x_train))),
    "mad": float(median_absolute_error(y_train, grid_search.predict(x_train))),
}

metrics_test = {
    "type": "metrics",
    "dataset": "test",
    "r2": float(r2_score(y_test, grid_search.predict(x_test))),
    "mse": float(mean_squared_error(y_test, grid_search.predict(x_test))),
    "mad": float(median_absolute_error(y_test, grid_search.predict(x_test))),
}


# Guardar las métricas en un archivo JSON
import json

    
with open('../files/output/metrics.json', 'w') as f:
    f.write(json.dumps(metrics_train) + "\n")
    f.write(json.dumps(metrics_test) + "\n")
